In [1]:
import jax
from jax import jit
import jax.numpy as jnp
import genjax
from genjax import gen
import equinox as eq
from genjax import inverse_gamma, normal, categorical, smc, Target
from tensorflow_probability.substrates import jax as tfp

from genjax import ChoiceMapBuilder as C
from genjax import Pytree, Weight, gen, pretty
from genjax._src.generative_functions.distributions.distribution import Distribution
from genjax.typing import PRNGKey
from jaxtyping import Array, Float, Integer
import time


tfd = tfp.distributions
key = jax.random.PRNGKey(0)
pretty()

2024-10-09 18:25:31.194699: W external/xla/xla/service/gpu/nvptx_compiler.cc:893] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version 12.6.68. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [2]:
@Pytree.dataclass
class NormalInverseGamma(Distribution):
    def random_weighted(self, key: PRNGKey, mu, l, a, b):
        ig = tfd.InverseGamma(concentration=a, scale=b)
        key, subkey = jax.random.split(key)
        precision = ig.sample(seed=subkey)
        ig_logp = ig.log_prob(precision)

        normal = tfd.Normal(loc=mu, scale=precision / l)
        key, subkey = jax.random.split(key)
        mu = normal.sample(seed=subkey)
        mu_logp = normal.log_prob(mu)
        
        retval = jnp.stack([mu, precision], axis=1)
        inv_logp = -jnp.sum(ig_logp) - jnp.sum(mu_logp)
        return inv_logp, retval

    def estimate_logpdf(self, key: PRNGKey, x, mu, l, a, b):
        mu_sampled = x[:,0]
        precision = x[:,1]
        ig = tfd.InverseGamma(concentration=a, scale=b)
        ig_logp = ig.log_prob(precision)
        normal = tfd.Normal(loc=mu, scale= precision/l)
        mu_logp = normal.log_prob(mu_sampled)
        return jnp.sum(ig_logp) + jnp.sum(mu_logp)

nig = NormalInverseGamma()

@gen
def model():
    x = nig(jnp.zeros(10), jnp.ones(10), jnp.ones(10), jnp.ones(10)) @ "x"
    return x

model.simulate(key, ())

StaticTrace(...)

In [3]:
@Pytree.dataclass
class Dirichlet(Distribution):
    def random_weighted(self, key:PRNGKey, alpha):
        dir = tfd.Dirichlet(concentration = alpha)
        probs = dir.sample(seed=key)
        inv_weight = -dir.log_prob(probs)
        return inv_weight, probs
    def estimate_logpdf(self, key:PRNGKey, x, alpha):
        dir = tfd.Dirichlet(concentration = alpha)
        return dir.log_prob(x)
dirichlet = Dirichlet()

@gen
def model():
    dist = dirichlet(jnp.ones((5,5))) @ "x"
    return dist

tr = model.simulate(key, ())

In [4]:
"""
A class to store DP samples and the corresponding beta values. 

Used in GEM to avoid floating point error
"""
@Pytree.dataclass
class DPSample(Pytree):
    betas: Array
    pi: Array
    def __init__(self, betas, pi):
        self.betas = betas
        self.pi = pi


In [35]:
def log1mexp(x):
    return jnp.where(-jnp.log(2) < x, jnp.log(-jnp.expm1(x)), jnp.log1p(-jnp.exp(x)))

print(-jnp.log(2))
a = jnp.array(-0.03244329398193646)
print(log1mexp(a))
print(jnp.log(1-jnp.exp(a)))

-0.6931472
-3.4444394
-3.44444


In [44]:

@Pytree.dataclass
class GEM(Distribution):
    C: int = Pytree.static(default=1)
    def __init__(self, C:int=10):
        self.C = jnp.asarray(C)
    def random_weighted(self, key: PRNGKey, alpha: Float):
        C = self.C
        sampler = tfd.Beta(concentration1 = jnp.array(alpha), concentration0=jnp.array(1.0))
        betas = sampler.sample(seed=key, sample_shape = C)
        inv_weight = -jnp.sum(sampler.log_prob(betas))

        betas = jnp.log(betas)
        betas_not = jax.vmap(log1mexp)(betas)
        print("betas ", betas)
        # prefix sum of betas
        logpi = jnp.zeros(C)
        for i in range(1,C):
            logpi = logpi.at[i].set(jnp.sum(betas_not[:i]))
        for i in range(C):
            logpi = logpi.at[i].set(logpi[i] + betas[i])

        return inv_weight, logpi

    def estimate_logpdf(self, key: PRNGKey, pi, alpha: Float):
        # assumes dist.pi corresponds to dist.betas
        sampler = tfd.Beta(concentration1 = jnp.array(alpha), concentration0 = jnp.array(1.0))
        def unfold(carry, pi):
            logbeta = pi - carry
            return carry + log1mexp(logbeta), logbeta

        _, betas = jax.lax.scan(unfold, 0.0, pi)
        print("betas ", betas)
        betas = jnp.exp(betas)
        weight = jnp.sum(sampler.log_prob(betas))
        return weight

gem = GEM(10)

@gen
def model():
    pi = gem(jnp.array(2.0)) @ "pi"
    return pi

# jax.jit(model.simulate)(key, ())
_, key = jax.random.split(key)
tr = model.simulate(key, ())
print("pi ", tr.retval)

print(gem.estimate_logpdf(key, tr.retval, jnp.array(2.0), ))

betas  Traced<ShapedArray(float32[10])>with<DynamicJaxprTrace(level=2/0)>
betas  [-0.59842694 -0.18138668 -0.49231285 -0.07199378 -0.8898936  -0.10840753
 -0.1964778  -0.04370651 -0.07381264 -1.4512422 ]
pi  [ -0.59842694  -0.9791739   -3.0865471   -3.610947    -7.0958033
  -6.8431363   -9.206779   -10.777844   -13.959982   -17.980316  ]
betas  [-0.59842694 -0.18138671 -0.49231315 -0.07199454 -0.889905   -0.10842705
 -0.19666767 -0.04477119 -0.09841633 -1.7513981 ]
2.497777


In [7]:
tr

StaticTrace(...)

In [8]:
@gen
def model():
    pi = gem(jnp.array(2.0)) @ "pi"
    return pi

simulate_jitted = jax.jit(jax.vmap(model.simulate, in_axes=(0,None)))
keys = jax.random.split(key, 1000)
simulate_jitted(keys, ())

StaticTrace(...)

In [9]:
# @Pytree.dataclass
# class GemByDirichlet(Distribution):
#     C: int = Pytree.static(default=1)
#     def __init__(self, C:int=10):
#         self.C = jnp.asarray(C)
#     def random_weighted(self, key: PRNGKey, concentration):
#         dir = tfd.Dirichlet(concentration=concentration)
#         probs = dir.sample(seed=key)
#         # invert betas
#         def unfold(carry, x):
#             beta = x / carry
#             return carry * (1-beta) , carry * beta
#         _, betas = jax.lax.scan(unfold, jnp.array(1.0), probs)
#         x = DPSample(betas, probs)
#         inv_weight = -dir.log_prob(probs)
#         return inv_weight, x

#     def estimate_logpdf(self, key: PRNGKey, x:DPSample, concentration):
#         # assumes dist.pi corresponds to dist.betas
#         dir = tfd.Dirichlet(concentration = concentration)
#         return dir.log_prob(x.pi)

# gbd = GemByDirichlet(10)
# @gen
# def model():
#     pi = gbd(jnp.ones((10,))) @ "pi"
#     return pi

# model.simulate(key, ())

In [10]:
@Pytree.dataclass
class MixtureModel(Distribution):
    def random_weighted(self, key, pi, categorical_probs):
        key_0, key_1 = jax.random.split(key, 2)
        cluster_dist = tfd.Categorical(pi)
        c = cluster_dist.sample(seed=key_0)
        c_logp = cluster_dist.log_prob(c)
        label_dist = tfd.Categorical(categorical_probs[c])
        y = label_dist.sample(seed=key_1)
        y_logp = label_dist.log_prob(y)
        return -c_logp-y_logp, (c,y)

    def estimate_logpdf(self, key:PRNGKey, x, pi, categorical_probs):
        c, y = x
        cluster_dist = tfd.Categorical(pi)
        label_dist = tfd.Categorical(categorical_probs[c])
        logp = cluster_dist.log_prob(c) + label_dist.log_prob(y)
        return logp

cmm = MixtureModel()

@genjax.repeat(n=100)
@gen 
def cluster(pi, probs):
    assignments = cmm(pi, probs) @ "assignments"
    return assignments

pi = jnp.ones(10) / 10
categorical_probs = jax.random.uniform(key, (10, 36, 19))
tr = cluster.simulate(key, (pi, categorical_probs,))
tr.get_choices()[0, "assignments"].unmask()

(<jax.Array(7, dtype=int32)>,
 <jax.Array int32(36,) [≥0, ≤17] zero:1 nonzero:35
   <Arrayviz rendering>
 >)

In [11]:
@genjax.repeat(n=100)
@gen 
def cluster(pi, probs):
    assignments = cmm(pi, probs) @ "assignments"
    return assignments

@gen
def model(data):
    C = 20
    n_features = data.shape[0]
    n_labels = 20
    pi = gem(jnp.array(2.0)) @ "pi"
    cluster_parameters = dirichlet(jnp.ones((C, n_features, n_labels))) @ "parameters"
    assignments = cluster(pi, cluster_parameters) @ "assignments"
    return assignments

simulate_jitted = jax.jit(model.simulate)
data = jax.random.uniform(key, (100, 36), minval=0, maxval=19)
simulate_jitted(key, (data,))

StaticTrace(...)

In [12]:
tr = simulate_jitted(key, ())
tr.get_choices()["assignments", 0, "assignments"]

TypeError: model() missing 1 required positional argument: 'data'

### Inference?

In [360]:
from genjax import ChoiceMapBuilder as C
import genspn.distributions 

key = jax.random.PRNGKey(0)
obs = C["assignments",0,"assignments"].set((jnp.array(2, dtype=jnp.int32), jnp.zeros((36,), dtype=jnp.int32)))
obs = tr.get_choices()
args = ()
model.importance(key, obs, args)

TypeError: true_fun and false_fun output must have identical types, got
('ShapedArray(float32[36])', 'DIFFERENT ShapedArray(float32[36]) vs. ShapedArray(float32[], weak_type=True)', ('ShapedArray(int32[])', 'ShapedArray(int32[36])')).

In [11]:
@gen
def rejuvenate_pi(assignments):
    C = 10
    cluster_counts = jnp.sum(jax.nn.one_hot(assignments, num_classes= C), axis=0)
    pi = gbd(cluster_counts) @ "pi"
    return pi

assignments, _ = tr.get_choices()["assignments", ..., "assignments"]

rejuvenate_pi.simulate(key, (assignments,))

StaticTrace(...)

In [12]:
@gen
def rejuvenate_parameters(assignments, data):
    C = 10
    n_features = 20
    # update posterior of hyperparameters
    one_hot_x = jax.nn.one_hot(data, num_classes=n_features)
    counts = jax.ops.segment_sum(one_hot_x, assignments, num_segments=C)
    concentration = jnp.ones((C, data.shape[1], n_features)) + counts
    parameters = dirichlet(concentration) @ "parameters"
    return parameters

assignments, _ = tr.get_choices()["assignments", ..., "assignments"]
data = jax.random.randint(key, (100, 36), minval=0, maxval=19)
rejuvenate_parameters.simulate(key, (assignments, data))

StaticTrace(...)

In [13]:
@gen
def rejuvenate_parameters(assignments, data):
    C = 10
    n_features = 20
    # update posterior of hyperparameters
    one_hot_x = jax.nn.one_hot(data, num_classes=n_features)
    counts = jax.ops.segment_sum(one_hot_x, assignments, num_segments=C)
    concentration = jnp.ones((C, data.shape[1], n_features)) + counts
    parameters = dirichlet(concentration) @ "parameters"
    return parameters

assignments, _ = tr.get_choices()["assignments", ..., "assignments"]
data = jax.random.randint(key, (100, 36), minval=0, maxval=19)
rejuvenate_parameters.simulate(key, (assignments, data))

StaticTrace(...)

In [22]:
@Pytree.dataclass
class ConjugatePrior(genjax.inference.Algorithm):
    def random_weighted(self, target: Target):
        # print(target)
        return jnp.array(1.0), target
    def estimate_logpdf(self, x, target: Target):
        return jnp.array(1.0)
    def estimate_normalizing_constant(self, target: Target):
        super().estimate_normalizing_constant(target)
    def estimate_reciprocal_normalizing_constant(self, key, target, latent_choices, w):
        return super().estimate_reciprocal_normalizing_constant(key, target, latent_choices, w)

conj = ConjugatePrior()

@genjax.gen
def model():
    x = genjax.normal(0.0, 1.0) @ "x"
    y = genjax.normal(x, 1.0) @ "y"
    return x

target = Target(model, (), C["y"].set(3.0))
conj.random_weighted(target)

(<jax.Array(1., dtype=float32, weak_type=True)>, Target(...))

In [32]:
import genspn.smc
@gen
def rejuvenate_assignments(label_logp):
    C = 10
    n_features = 20
    # genjax evaluate

    # assignments = jax.random.categorical(key, log_score, axis=-1).astype(int)
    # categorical()
    # return assignments

assignments, _ = tr.get_choices()["assignments", ..., "assignments"]
parameters = tr.get_choices()["parameters"]
mask = genspn.smc.make_log_likelihood_mask(assignments, 10)
# categorical.estimate_logpdf(key, parameters)
# logprobs = jax.vmap(jax.scipy.stats.dirichlet.logpdf)(jnp.exp(x.logprobs), dist.alpha)
# return jnp.sum(logprobs)
# log_likelihoods = jax.vmap(jax.vmap(genspn.distributions.logpdf, in_axes=(0, None)), in_axes=(None, 0))(parameters, data)
# log_likelihoods = log_likelihoods + mask
# log_score = log_likelihoods + jnp.log(pi)
# data = jax.random.randint(key, (100, 36), minval=0, maxval=19)
# rejuvenate_parameters.simulate(key, (assignments, data))

0.99999994


In [43]:
train, test = load_huggingface("AutoML/soybean")

shape: (307, 36)
┌────────────────┬──────┬─────────────┬────────┬───┬───────────────┬───────────┬────────────┬──────┐
│ class          ┆ date ┆ plant_stand ┆ precip ┆ … ┆ seed_discolor ┆ seed_size ┆ shriveling ┆ root │
│ ---            ┆ ---  ┆ ---         ┆ ---    ┆   ┆ ---           ┆ ---       ┆ ---        ┆ ---  │
│ str            ┆ str  ┆ str         ┆ str    ┆   ┆ str           ┆ str       ┆ str        ┆ str  │
╞════════════════╪══════╪═════════════╪════════╪═══╪═══════════════╪═══════════╪════════════╪══════╡
│ phytophthora_r ┆ 2    ┆ 1           ┆ 1      ┆ … ┆ null          ┆ null      ┆ null       ┆ 1    │
│ ot             ┆      ┆             ┆        ┆   ┆               ┆           ┆            ┆      │
│ frog_eye_leaf_ ┆ 3    ┆ 0           ┆ 2      ┆ … ┆ 0             ┆ 0         ┆ 0          ┆ 0    │
│ spot           ┆      ┆             ┆        ┆   ┆               ┆           ┆            ┆      │
│ phytophthora_r ┆ 1    ┆ 1           ┆ 2      ┆ … ┆ 0             ┆ 0    

/home/ubuntu/docs/GenSPN/src/genspn/io.py:22: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  df = df.cast(pl.Categorical)


In [13]:
def f(key, logp):
    return jax.random.categorical(key, logp)

key = jax.random.PRNGKey(0)
logp = jnp.ones((10000, 10))
f_jitted = jax.jit(f)
f_jitted(key, logp)
start = time.process_time()
f_jitted(key, logp).block_until_ready()
print((time.process_time() - start)*1000)

0.3081829999995733
